In [1]:
import torch
from dual_llama import LlamaForCausalLM

from dual_phi import PhiForCausalLM
from transformers import AutoTokenizer, AutoConfig

# model_path = "/apdcephfs_qy3/share_1594716/bingzhe/pretrained/Llama-2-7b-hf/"
# model = LlamaForCausalLM.from_pretrained(model_path, device_map="cpu")


model_path = "/apdcephfs_qy3/share_1594716/bingzhe/pretrained/phi-1_5"

config = AutoConfig.from_pretrained(model_path)

model = PhiForCausalLM.from_pretrained(model_path, device_map="cpu")

Some weights of PhiForCausalLM were not initialized from the model checkpoint at /apdcephfs_qy3/share_1594716/bingzhe/pretrained/phi-1_5 and are newly initialized: ['model.channel_emb.channel_embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
model_new = PhiForCausalLM.from_pretrained("phi", device_map="cpu")

for name, param in model_new.named_parameters():
    if param.requires_grad:
        print(f"Parameter: {name}, requires_grad: {param.requires_grad}")
    else:
        print(f"Parameter: {name}, requires_grad: {param.requires_grad} (not tracking gradients)")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Parameter: model.embed_tokens.weight, requires_grad: True
Parameter: model.layers.0.self_attn.q_proj.weight, requires_grad: True
Parameter: model.layers.0.self_attn.q_proj.bias, requires_grad: True
Parameter: model.layers.0.self_attn.k_proj.weight, requires_grad: True
Parameter: model.layers.0.self_attn.k_proj.bias, requires_grad: True
Parameter: model.layers.0.self_attn.v_proj.weight, requires_grad: True
Parameter: model.layers.0.self_attn.v_proj.bias, requires_grad: True
Parameter: model.layers.0.self_attn.dense.weight, requires_grad: True
Parameter: model.layers.0.self_attn.dense.bias, requires_grad: True
Parameter: model.layers.0.mlp.fc1.weight, requires_grad: True
Parameter: model.layers.0.mlp.fc1.bias, requires_grad: True
Parameter: model.layers.0.mlp.fc2.weight, requires_grad: True
Parameter: model.layers.0.mlp.fc2.bias, requires_grad: True
Parameter: model.layers.0.input_layernorm.weight, requires_grad: True
Parameter: model.layers.0.input_layernorm.bias, requires_grad: True
Pa

In [18]:
tokenizer = AutoTokenizer.from_pretrained(model_path, padding_sided="left")
tokenizer.pad_token_id = 0
tokenizer.eos_token_id = 2
tokenizer.padding_side = "left"


prompt = ['''def remove_non_ascii(s: str) -> str:
    """ <FILL_ME>
    return result
''', "hello, my name is Jack"]

input_ids = tokenizer(prompt, return_tensors="pt", padding=True)

In [19]:
input_ids

{'input_ids': tensor([[ 4299,  4781,    62, 13159,    62,   292,   979,    72,     7,    82,
            25,   965,     8,  4613,   965,    25,   198, 50284, 37811,  1279,
            37,  8267,    62, 11682,    29,   198, 50284,  7783,  1255,   198],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0, 31373,    11,   616,  1438,   318,  3619]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 1, 1, 1, 1, 1]])}

In [2]:
model

NameError: name 'model' is not defined

In [3]:
model.save_pretrained("phi")

In [21]:
output = model.generate(
    **input_ids,
    max_new_tokens=105,
    pad_token_id=0,
    do_sample=False,
    max_length=40
)
# model(**input_ids)

output

tensor([[ 4299,  4781,    62, 13159,    62,   292,   979,    72,     7,    82,
            25,   965,     8,  4613,   965,    25,   198, 50284, 37811,  1279,
            37,  8267,    62, 11682,    29,   198, 50284,  7783,  1255,   198,
            25, 50284,   198, 37811, 50284,   198,  7783, 50284,   705,  7783],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0, 31373,    11,   616,  1438,   318,  3619,
           340,    13,   338,   314,   257,  1101,  1643,  9675,   286,   345]])

In [25]:
output
tokenizer.decode(output[0])

'def remove_non_ascii(s: str) -> str:\n    """ <FILL_ME>\n    return result\n:    \n"""    \nreturn    \'return'

In [24]:
output
tokenizer.decode(output[1])

"!!!!!!!!!!!!!!!!!!!!!!!!hello, my name is Jack it.'s I a'm bit glad of you"

In [10]:
unsqueeze_dim = 1
position_ids = 30
cos = torch.rand([1, 30, 128]) * 0.3
sin = torch.rand([1, 30, 128]) 
q = torch.rand([1, 32, 30, 128])
k = torch.rand([1, 32, 30, 128])
position_ids = torch.arange(30)


def rotate_half(x):
    """Rotates half the hidden dims of the input."""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)

if position_ids is not None:
    # Assuming position_ids is of shape [batch_size, seq_len]
    # We need to create a new position_ids that groups adjacent tokens
    new_position_ids = position_ids // 2  # Grouping adjacent tokens
else:
    # If position_ids is not provided, we create a default one
    seq_len = q.size(2)  # Assuming q has shape [batch_size, heads, seq_len, head_dim]
    new_position_ids = torch.arange(seq_len, device=q.device) // 2  # Grouping adjacent tokens

# Get the cosine and sine values for the new position ids
cos = cos[:, new_position_ids, :]
sin = sin[:, new_position_ids, :]

cos = cos.unsqueeze(unsqueeze_dim)
sin = sin.unsqueeze(unsqueeze_dim)

q_embed = (q * cos) + (rotate_half(q) * sin)
k_embed = (k * cos) + (rotate_half(k) * sin)

In [11]:
cos.shape

torch.Size([1, 1, 30, 128])

In [12]:
from transformers.configuration_utils import PretrainedConfig
from transformers.modeling_rope_utils import rope_config_validation

In [13]:
import numpy as np

result_1 = 500 ** 50
result_2 = 1000 ** 20

result_1 < result_2

False

In [14]:
import torch

def precompute_freqs_cis(dim: int, end: int, theta: float = 10000.0):
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
    t = torch.arange(end, device=freqs.device)  # type: ignore
    freqs = torch.outer(t, freqs).float()  # type: ignore
    freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    return freqs_cis

In [15]:
import torch

dim = 4096
end = 100
theta = 10000
freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
t = torch.arange(end, device=freqs.device)  # type: ignore
freqs = torch.outer(t, freqs).float()  # type: ignore
freqs_cis = torch.polar(torch.ones_like(freqs), freqs) 

In [16]:
torch.arange(0, dim, 2)[: (dim // 2)].float()

tensor([0.0000e+00, 2.0000e+00, 4.0000e+00,  ..., 4.0900e+03, 4.0920e+03,
        4.0940e+03])

In [17]:
(torch.arange(0, dim, 2)[: (dim // 2)].float() / dim).shape

torch.Size([2048])

In [19]:
dim // 2

2048